In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
from multiprocessing import Pool
import csv

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
browser = webdriver.Chrome()
browser.get('https://www.douyu.com/60937')

In [ ]:
cast_name = browser.find_element_by_xpath('//a[@class="zb-name"]/h1').text
date = time.strftime('%m-%d')

data_file = 'data_in_csv/%s-%s.csv' % (cast_name, date)
log_file = 'log/%s-%s.txt' % (cast_name, date)

with open(log_file, 'a+', encoding='utf-8') as log:
    log.write(time.strftime('>>>>logging time:%Y/%m/%d-%H:%M:%S<<<<<\n'))
    
while True:
    s = time.time()
    chunks = re.findall('<li class="jschartli hy-chat(.*?)</li>', browser.page_source, re.S)
    
    node = browser.find_elements_by_xpath('//ul[@class="c-list"]/li[@class="jschartli hy-chat  "]')[-1]
    wait = WebDriverWait(browser, 3600)
    count = 0
    
    for c in chunks:
        level = re.findall('data-level="(.*?)" ', c, re.S)[0]
        if re.findall('<i>(.*?)</i>',c, re.S):
            fans = re.findall('<i>(.*?)</i>', c, re.S)[0]
        else:
            fans = '无粉丝牌'
        name = re.findall('data-noble=".*?">(.*?)：<', c, re.S)[0]
        dm = re.findall('chatid=.*?">(.*?)</span', c, re.S)[0]
        count += 1
        
        with open(data_file, 'a+', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([name, level, fans, dm])
            
    e = time.time()        
    print('InsertTime:', round(e-s, 4), end=' , ')
    print('AddNumber:  ', count, end=' , ')
    s_ = time.time()
    wait.until(EC.staleness_of(node))
    e_ = time.time()
    print('WaitingTime:', round(e_-s_,4))
    
    with open(log_file, 'a+', encoding='utf-8') as log:
        log.write('InsertTime:  %8.4f , ' % (e - s))
        log.write('AddNumber:   %8d , '   % (count))
        log.write('WaitingTime: %8.2f\n' % (e_-s_))